In [12]:
# Import libraries

import re
import string
import nltk
import numpy as np
from nltk.corpus import wordnet 
from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('stopwords')
from nltk.stem import PorterStemmer, WordNetLemmatizer
nltk.download("wordnet")
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk import sent_tokenize
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('omw-1.4')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\sorok\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\sorok\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\sorok\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\sorok\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\sorok\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\omw-1.4.zip.


True

In [5]:
# Load the text file

filename = 'alice_in_wonderland.txt'
with open(filename, encoding='utf-8') as f:
    alice_in_wonderland = f.read()
alice_in_wonderland[:500]

'The Project Gutenberg eBook of Alice’s Adventures in Wonderland, by Lewis Carroll\n\nThis eBook is for the use of anyone anywhere in the United States and\nmost other parts of the world at no cost and with almost no restrictions\nwhatsoever. You may copy it, give it away or re-use it under the terms\nof the Project Gutenberg License included with this eBook or online at\nwww.gutenberg.org. If you are not located in the United States, you\nwill have to check the laws of the country where you are located'

In [7]:
# Selecting the text

textfile = alice_in_wonderland.split('CHAPTER I.')[2]
textfile = textfile.split('THE END')[0]
textfile[:500]

'\nDown the Rabbit-Hole\n\n\nAlice was beginning to get very tired of sitting by her sister on the\nbank, and of having nothing to do: once or twice she had peeped into\nthe book her sister was reading, but it had no pictures or\nconversations in it, “and what is the use of a book,” thought Alice\n“without pictures or conversations?”\n\nSo she was considering in her own mind (as well as she could, for the\nhot day made her feel very sleepy and stupid), whether the pleasure of\nmaking a daisy-chain would be w'

In [9]:
# Function to preprocess a text

def preprocess_textfile(input_text):
    formatted_text = input_text.lower()
    formatted_text = re.sub(r"won\'t", "will not", formatted_text)
    formatted_text = re.sub(r"can\'t", "can not", formatted_text)
    formatted_text = re.sub(r"n\'t", " not", formatted_text)
    formatted_text = re.sub(r"\'re", " are", formatted_text)
    formatted_text = re.sub(r"\'s", " is", formatted_text)
    formatted_text = re.sub(r"\'d", " would", formatted_text)
    formatted_text = re.sub(r"\'ll", " will", formatted_text)
    formatted_text = re.sub(r"\'t", " not", formatted_text)
    formatted_text = re.sub(r"\'ve", " have", formatted_text)
    formatted_text = re.sub(r"\'m", " am", formatted_text)
    formatted_text = re.sub(r"[^\w\s]", "", formatted_text)
    formatted_text = re.sub("_", " ", formatted_text)
    text_tokens = formatted_text.split()
    stop_words = set(stopwords.words('english'))
    sw_tokens = [token for token in text_tokens if not token in stop_words]
    lemmatizer = WordNetLemmatizer()
    lemm_tokens = [lemmatizer.lemmatize(token) for token in sw_tokens]
    lemm_tokens = [lemmatizer.lemmatize(token, "v") for token in sw_tokens]    # For verbs
    formatted_text = " ".join(lemm_tokens)
    return formatted_text

In [22]:
# Task 3: Find 10 most important words from each chapter

all_chapters = textfile.split('CHAPTER ')
tf_idf = TfidfVectorizer(stop_words='english')
tf_idf.fit_transform(all_chapters)
print("10 MOST IMPORTANT WORDS FROM EACH CHAPTER.")
chapter_num = 1
for chapter in all_chapters:
    clear_chapter = preprocess_textfile(chapter)
    clear_chapter = re.sub(r'alice', '', clear_chapter)  # No Alice for this task
    response = tf_idf.transform([clear_chapter])
    all_words = np.array(tf_idf.get_feature_names())
    tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]
    top_ten = all_words[tfidf_sorting][:10]
    print("CHAPTER ", chapter_num, ":")
    for i in top_ten:
        print(i, end=" ")
    print("\n")
    chapter_num += 1

10 MOST IMPORTANT WORDS FROM EACH CHAPTER.
CHAPTER  1 :
think eat say little bat fall key try wonder happen 

CHAPTER  2 :
mouse say ill pool little swim think cat dear fan 

CHAPTER  3 :
say mouse dodo prize bird lory ill dry know thimble 

CHAPTER  4 :
grow little window rabbit say run fan puppy hear come 

CHAPTER  5 :
say caterpillar pigeon serpent egg youth bite try think size 

CHAPTER  6 :
say cat footman baby mad duchess sneeze pig grin think 

CHAPTER  7 :
say hatter dormouse march hare twinkle draw remark time know 

CHAPTER  8 :
queen say soldier gardeners look king hedgehog cat head executioner 

CHAPTER  9 :
say turtle mock gryphon duchess moral queen think sigh remark 

CHAPTER  10 :
turtle mock gryphon say dance beautiful soup join repeat lobster 

CHAPTER  11 :
king hatter say court dormouse witness begin queen officer look 

CHAPTER  12 :
say king jury write dream queen sister mean rabbit jurymen 



In [31]:
# Task 4: Find the Top 10 most used verbs in sentences with Alice

sents = sent_tokenize(textfile)
alice_sents = []
verbs = ''
for sentence in sents:
    clearSentence = preprocess_textfile(sentence)
    clearSentence = re.sub(r'king', '', clearSentence)  # It's a noun, but considered as verb
    if 'alice' in clearSentence:
        alice_sents.append(clearSentence)
    else:
        continue
for sentence in alice_sents:
    s = nltk.pos_tag(sentence.split())
    for w in s:
        if 'VB' in w[1]:
            verbs += w[0] + ' '
        else:
            continue
response = tf_idf.transform([verbs])
feature_array = np.array(tf_idf.get_feature_names())
tfidf_sorting = np.argsort(response.toarray()).flatten()[::-1]
print("10 MOST USED VERBS IN SENTENCES WITH ALICE:")
for i in feature_array[tfidf_sorting][:10]:
        print(i, end=" ")

10 MOST USED VERBS IN SENTENCES WITH ALICE:
say think know run come make begin happen look tell 